In [1]:
#!/usr/bin/env conda run -n pymc3venv python

import sys
print(sys.executable)    ### Shoul be this C:\ProgramData\Anaconda3_1\envs\pymc3venv\python.exe

C:\ProgramData\Anaconda3_1\envs\pymc3venv\python.exe


In [7]:
!conda install --yes --prefix {sys.prefix} keras

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3_1\envs\pymc3venv

  added / updated specs:
    - keras


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    keras-2.3.1                |                0           6 KB
    keras-base-2.3.1           |           py37_0         585 KB
    ------------------------------------------------------------
                                           Total:         591 KB

The following NEW packages will be INSTALLED:

  keras              pkgs/main/win-64::keras-2.3.1-0
  keras-base         pkgs/main/win-64::keras-base-2.3.1-py37_0




keras-base-2.3.1     | 585 KB    |            |   0% 
keras-base-2.3.1     | 585 KB    | 2          |   3% 
keras-base-2.3.1     | 585 KB    | ########## | 100% 

keras-2.3.1          | 6 KB      |            |   0% 
keras-2.3.1          | 6 KB      | #########

In [10]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:80% !important; }</style>"))
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import percentileofscore
from scipy import stats
from sklearn.model_selection import train_test_split
import h5py
import tensorflow as tf
from sklearn.metrics import classification_report

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

In [3]:
# load data

dfg = pd.read_hdf('dfg3.h5', key='losses')
df = dfg.copy()
df.columns = ['spv','N160','PI','LL','wn','CUvane','Plim','OCR','CuTx']

In [4]:
# Create test train data non need to normalise

X_data = df.drop('CuTx', axis=1)
y_data = df['CuTx']

# No need for this now
X_train,X_test,y_train,y_test = train_test_split(X_data,y_data,test_size=0.7,random_state=101)

In [9]:
# create model from class function
model = Sequential()

#get number of columns in training data
n_cols = X_train.shape[1]

#Add model layers 2 hidden layer and an output layer
#Dense is a standard layer type that works for most cases
#We have 10 nodes in each of our input layers (higher the better but slower)
#‘Activation’ is a function allowing for nonlinear relationships
#The activation function we will be using is ReLU or Rectified Linear
#The first layer needs an input shape; n° of rows and columns in the input. 
#Note: There is nothing after the comma: indicate any amount of rows

model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))         

In [12]:
#Compiling the model takes two parameters: optimizer and loss
#Adam is generally a good optimizer
#compile model using mse as a measure of model performance

model.compile(optimizer='adam', loss='mean_squared_error')

In [13]:
#Training the model
#training data,target data,validation split, epochs and callbacks
#The validation split training and testing
#The number of epochs is the number of times the model will cycle through the data
#Early stopping will stop the model from training before the number of epochs is reached if the model stops improving

early_stopping_monitor = EarlyStopping(patience=3)

#Train model

model.fit(X_data, y_data, validation_split=0.3, epochs=30,
         callbacks=[early_stopping_monitor])






Train on 323 samples, validate on 139 samples
Epoch 1/30
323/323 [==============================] - 0s 719us/step - loss: 26447.7708 - val_loss: 49177.2712
Epoch 2/30
323/323 [==============================] - 0s 43us/step - loss: 13270.7932 - val_loss: 32510.9992
Epoch 3/30
323/323 [==============================] - 0s 50us/step - loss: 6599.3520 - val_loss: 22705.4405
Epoch 4/30
323/323 [==============================] - 0s 51us/step - loss: 4688.4609 - val_loss: 18685.4084
Epoch 5/30
323/323 [==============================] - 0s 41us/step - loss: 4613.3806 - val_loss: 18024.3984
Epoch 6/30
323/323 [==============================] - 0s 37us/step - loss: 4478.7293 - val_loss: 18454.4114
Epoch 7/30
323/323 [==============================] - 0s 40us/step - loss: 4319.2307 - val_loss: 18503.7741
Epoch 8/30
323/323 [==============================] - 0s 40us/step - loss: 4218.8969 - val_loss: 18264.0165


In [15]:
#example on how to use our newly trained model on how to make predictions on unseen data (we will pretend our new data is saved in a dataframe called 'test_X').

test_y_predictions = model.predict(X_test)

In [41]:
#Train a new model with increasing model capacity

model_mc = Sequential()

model_mc.add(Dense(200,activation='relu',input_shape=(n_cols,)))
model_mc.add(Dense(200,activation='relu'))
model_mc.add(Dense(200,activation='relu'))
model_mc.add(Dense(1))

model_mc.compile(optimizer='adam', loss='mean_squared_error')

model_mc.fit(X_data, y_data, validation_split=0.3, epochs=30, callbacks=[early_stopping_monitor])


Train on 323 samples, validate on 139 samples
Epoch 1/30
323/323 [==============================] - 0s 918us/step - loss: 3794.8878 - val_loss: 2429.4504
Epoch 2/30
323/323 [==============================] - 0s 106us/step - loss: 2040.2059 - val_loss: 938.5332
Epoch 3/30
323/323 [==============================] - 0s 116us/step - loss: 1208.4566 - val_loss: 996.7022
Epoch 4/30
323/323 [==============================] - 0s 123us/step - loss: 751.7277 - val_loss: 877.8318
Epoch 5/30
323/323 [==============================] - 0s 121us/step - loss: 441.1257 - val_loss: 1602.0813
Epoch 6/30
323/323 [==============================] - 0s 103us/step - loss: 357.1865 - val_loss: 1671.2849
Epoch 7/30
323/323 [==============================] - 0s 120us/step - loss: 323.2842 - val_loss: 1447.5025
